### 2023년 네이버뉴스 크롤링 

In [ ]:
# 이렇게하려고했으나 1~6페이지 크롤링 후 8페이지로 넘어가고 11,15,20페이지 크롤링함.. 왜지 대체왜지

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

url = 'https://search.naver.com/search.naver?where=news&query=%EC%9A%B0%EC%9A%B8%EC%A6%9D&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.12.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20231231&is_sug_officeid=0&office_category=0&service_area=0'

options = Options()
# Chrome 브라우저를 실행할 때 창 크기를 최대화하는 옵션을 추가
options.add_argument("--start-maximized")
# "detach"라는 실험적인 옵션을 추가하여 브라우저 창을 분리 가능하도록 설정
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options = options)
driver.get(url)
time.sleep(2)

title = []
text = []

for i in range(1, 11):
    page = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/div[2]/div/div/a[{i}]')
    page.click()
    time.sleep(2)
    for j in range(1, 11):
        try:
            title_a = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/section[1]/div/div[2]/ul/li[{j}]/div/div/div[2]/a[2]')
        except:
            title_a = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/section[1]/div/div[2]/ul/li[{j}]/div/div/div[2]/a')
        title_text = title_a.text
        title.append(title_text)
#         print(title_text)
        time.sleep(2)
        
        text_a = driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div/div[1]/section[1]/div/div[2]/ul/li[{j}]/div/div/div[2]/div/div/a')
        text_text = text_a.text
        text.append(text_text)
        print(text_text)
        print('\n')
        time.sleep(2)
    time.sleep(2)
    
df = pd.DataFrame({'Title': title, 'Text': text})
df.to_csv(r'C:\Users\soobin\coding_excuse\crawling_soobin.csv',index=False, encoding='utf-8-sig')        

In [72]:
# 예림언니 코드로 한것
from bs4 import BeautifulSoup 
import requests
import pandas as pd

url = 'https://search.naver.com/search.naver?where=news&query=%EC%9A%B0%EC%9A%B8%EC%A6%9D&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.12.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20231231&is_sug_officeid=0&office_category=0&service_area=0'
res = requests.get(url)

def get_news_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    page_titles = []
    page_texts = []

    # 제목과 관련된 텍스트 수집
    for a_tag in soup.find_all('a', class_='news_tit'):
        page_titles.append(a_tag.get_text())

    # 본문과 관련된 텍스트 수집
    for a_tag in soup.find_all('a', class_='api_txt_lines dsc_txt_wrap'):
        page_texts.append(a_tag.get_text())

    return page_titles, page_texts

base_url = 'https://search.naver.com/search.naver?where=news&query=%EC%9A%B0%EC%9A%B8%EC%A6%9D&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.12.31&start={}'

titles = []
texts = []

# 10 페이지까지 반복
for page in range(1, 11):
    start = (page - 1) * 10 + 1
    page_url = base_url.format(start)
    page_titles, page_texts = get_news_from_page(page_url)
    titles.extend(page_titles)
    texts.extend(page_texts)
    
# 데이터프레임 생성 및 CSV 파일로 저장
df = pd.DataFrame({'title': titles, 'text': texts})
csv_file_path = 'news_data_sb.csv'
df.to_csv(csv_file_path, index=False, encoding = 'utf-8-sig')
